In [ ]:
import os
import random
import numpy as np
import sys
import torch
import pprint

root = '/'
pprint.pprint(sys.path)

sys.path.append(import_path)
pprint.pprint(sys.path)

In [ ]:
import os
import sys
import pprint

import_path = root + 'pigvae_all'
sys.path.append(import_path)
pprint.pprint(sys.path)

import_path1 = root + 'pigvae_all/pigvae'
sys.path.append(import_path1)
pprint.pprint(sys.path)

['/content',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/usr/local/lib/python3.10/dist-packages/setuptools/_vendor',
 '/root/.ipython',
 '/content/drive/MyDrive/graph_lda/pigvae_all/pigvae']


In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.utils.data.distributed import DistributedSampler
import random
#import pytorch_lightning as pl
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
import networkx as nx
from networkx.algorithms.shortest_paths.dense import floyd_warshall_numpy

from networkx.generators.random_graphs import *
from networkx.generators.ego import ego_graph
from networkx.generators.geometric import random_geometric_graph

In [ ]:
loaded_data = torch.load(root + 'dataset/train_dataset/zinc_gdata/tensor_data.pkl')

# 各テンソルにアクセス
node_features = loaded_data['node_features']
edge_features = loaded_data['edge_features']
mask = loaded_data['mask']
props = loaded_data['props']

<ipython-input-11-9f298e5238d8>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_data = torch.load(root + 'zinc_gdata/tensor_data.pkl')


In [ ]:
num_node_f, num_edge_f = 43, 6

In [ ]:
node_features.shape, edge_features.shape, mask.shape, props.shape

(torch.Size([75000, 26, 43]),
 torch.Size([75000, 26, 26, 13]),
 torch.Size([75000, 26]),
 torch.Size([75000]))

In [ ]:
train_dataset = torch.utils.data.TensorDataset(node_features[0:50000], edge_features[0:50000], mask[0:50000], props[0:50000])
val_dataset = torch.utils.data.TensorDataset(node_features[50000:60000], edge_features[50000:60000], mask[50000:60000], props[50000:60000])
test_dataset = torch.utils.data.TensorDataset(node_features[60000:70000], edge_features[60000:70000], mask[60000:70000], props[60000:70000])

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=20, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=20, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=20, shuffle=False)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
hparams = {
    "vae":False,
    "kld_loss_scale":0.01,
    "perm_loss_scale":0.1,
    "property_loss_scale":0.5,
    "num_node_features":num_node_f,
    "num_edge_features":6+num_edge_f+1,
    "emb_dim": 90,
    'graph_encoder_hidden_dim': 256,
    'graph_encoder_k_dim': 64,
    'graph_encoder_v_dim': 64,
    'graph_encoder_num_heads': 16,
    'graph_encoder_ppf_hidden_dim': 512,
    'graph_encoder_num_layers': 16,
    'graph_decoder_hidden_dim': 256,
    'graph_decoder_k_dim': 64,
    'graph_decoder_v_dim': 64,
    'graph_decoder_num_heads': 16,
    'graph_decoder_ppf_hidden_dim': 512,
    'graph_decoder_num_layers': 16,
    "graph_decoder_pos_emb_dim": 64,
    'property_predictor_hidden_dim': 3,
    'num_properties': 1
}

model_save_dir = root + 'save_models/zinc/pig-beta-ae_models/'

if not os.path.exists(model_save_dir):
    os.makedirs(model_save_dir)

In [ ]:
from pigvae.models import GraphAE

model = GraphAE(hparams).to(device)

In [ ]:
from pigvae.synthetic_graphs.critic_zinc import Critic

critic = Critic(hparams, device)

In [ ]:
lr = 0.00005

optimizer = torch.optim.Adam(model.parameters(), lr=0.00005, betas=(0.9, 0.98))
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.9)

# Training

In [ ]:
train_loss_list = []
val_loss_list = []

def val_preds(epoch):

    model.eval()

    sum_errors = 0

    for batch_idx, batch_data in enumerate(val_loader):

        with torch.no_grad():

            if torch.cuda.is_available():
                node_features, edge_features, mask, props = batch_data
                node_features, edge_features, mask, props = node_features.to(device), edge_features.to(device), mask.to(device), props.to(device)

            graph_pred, perm, mu, logvar = model(node_features, edge_features, mask, training=True, tau=1.0)
            node_logits, edge_logits, pred_props = graph_pred

            vae_loss = critic(node_features, edge_features, props, node_logits, edge_logits, pred_props, mask, perm, mu, logvar)

            batch_vae_loss = vae_loss['node_loss'] * 10 + vae_loss['edge_loss'] * 10 + vae_loss['perm_loss'] * 0.1

            if batch_idx % 10 == 0:
                print(vae_loss)
                print("Epoch (val) : ", epoch, "  batch (val) : ", batch_idx , "Loss sum : ", batch_vae_loss.item())

            sum_errors = sum_errors + vae_loss['node_loss'].data.item() * 10 + vae_loss['edge_loss'].data.item() * 10

            del batch_vae_loss
            torch.cuda.empty_cache()

    return sum_errors

In [ ]:
import datetime

# テキストファイルのパスを指定
log_path = model_save_dir + 'log.txt'

f = open(log_path, 'w')
f.write('train -- logs')
f.close()

def logging_txt(epoch):
    dt_now = datetime.datetime.now()

    # ファイルを読み込んで、末尾の行を取得
    with open(log_path, 'r') as file:
        lines = file.readlines()
        last_line = lines[-1].strip()

    # 末尾の行に追記する文字列を指定
    append_string = "epoch " + str(epoch) + " : " + str(dt_now)

    # ファイルに追記
    with open(log_path, 'a') as file:
        file.write('\n' + append_string)

In [ ]:
num_epochs = 100
best_loss = 10000000

for epoch in range(num_epochs):
    model.train()

    sum_errors = 0

    for batch_idx, batch_data in enumerate(train_loader):

        node_features, edge_features, mask, props = batch_data
        node_features, edge_features, mask, props = node_features.to(device), edge_features.to(device), mask.to(device), props.to(device)

        optimizer.zero_grad()

        graph_pred, perm, mu, logvar = model(node_features, edge_features, mask, training=True, tau=1.0)
        node_logits, edge_logits, pred_props = graph_pred

        vae_loss = critic(node_features, edge_features, props, node_logits, edge_logits, pred_props, mask, perm, mu, logvar)

        batch_vae_loss = vae_loss['node_loss'] * 10 + vae_loss['edge_loss'] * 10 + vae_loss['perm_loss'] * 0.1

        if batch_idx % 100 == 0:
            print("Epoch : ", epoch, " Batch : ", batch_idx, " Loss : ", batch_vae_loss.data.item())
            print(vae_loss)

        sum_errors = sum_errors + vae_loss['node_loss'].data.item() * 10 + vae_loss['edge_loss'].data.item() * 10

        batch_vae_loss.backward()

        optimizer.step()

        del batch_vae_loss
        torch.cuda.empty_cache()

    lr_scheduler.step()

    train_loss_list.append(sum_errors / len(train_loader))
    val_loss = val_preds(epoch) / len(val_loader)
    val_loss_list.append(val_loss)

    np.save(model_save_dir + "train_loss_hist", np.array(train_loss_list))
    np.save(model_save_dir + "val_loss_hist", np.array(val_loss_list))

    logging_txt(epoch)

    if val_loss < best_loss:
        save_model_path = model_save_dir + "pigvae_best_model" + ".pt"
        torch.save(model, save_model_path)

        optimizer_path = model_save_dir + "opt_best_model" + ".pt"
        torch.save(optimizer.state_dict(), optimizer_path)

        best_loss = val_loss

Streaming output truncated to the last 5000 lines.
{'edge_loss': tensor(0.0480, device='cuda:0', grad_fn=<DivBackward0>), 'node_loss': tensor(-0.0049, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'perm_loss': tensor(1.6011, device='cuda:0', grad_fn=<AddBackward0>), 'property_loss': tensor(511.0961, device='cuda:0', grad_fn=<MseLossBackward0>)}
Epoch :  94  Batch :  340  Loss :  0.6907867789268494
{'edge_loss': tensor(0.0517, device='cuda:0', grad_fn=<DivBackward0>), 'node_loss': tensor(0.0021, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'perm_loss': tensor(1.5268, device='cuda:0', grad_fn=<AddBackward0>), 'property_loss': tensor(458.8059, device='cuda:0', grad_fn=<MseLossBackward0>)}
Epoch :  94  Batch :  350  Loss :  0.6263706088066101
{'edge_loss': tensor(0.0510, device='cuda:0', grad_fn=<DivBackward0>), 'node_loss': tensor(-0.0042, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'perm_loss': tensor(